In [19]:
from imports import *
from preprocessing import *

In [2]:
import re

def expand_contractions(text):
    # Define the contractions pattern
    contractions_pattern = re.compile('({})'.format('|'.join(CONTRACTIONS.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)
    
    def replace(match):
        # Get the contraction from the match object
        contraction = match.group(0)
        
        # Use the CONTRACTIONS dictionary to get the expanded form
        expanded = CONTRACTIONS.get(contraction.lower())
        
        # Capitalize the expanded form if the contraction was capitalized
        if contraction[0].isupper():
            expanded = expanded.capitalize()
        
        return expanded
    
    # Replace the contractions in the text using the replace function
    expanded_text = contractions_pattern.sub(replace, text)
    
    return expanded_text

# Define a dictionary of common contractions and their expanded forms
CONTRACTIONS = {
    "ain't": "am not",
    "aren't": "are not / am not",
    "can't": "cannot",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'll": "he shall / he will",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'll": "how will",
    "how's": "how has / how is",
    "I'd": "I had / I would",
    "I'll": "I shall / I will",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'll": "it shall / it will",
    "it's": "it has / it is",
    "let's": "let us",
    "might've": "might have",
    "mightn't": "might not",
    "must've": "must have",
    "mustn't": "must not",
    "shan't": "shall not",
    "she'd": "she had / she would",
    "she'll": "she shall / she will",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "that's": "that has / that is",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'll": "they shall / they will",
    "they're": "they are",
    "they've": "they have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'll": "we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "where's": "where has"
}

text = "What've you been ? I ain't waiting for you"
new_text = expand_contractions(text)
print(new_text)

What have you been ? I am not waiting for you


In [27]:
df = pd.read_csv("final_df.csv")
df = df.dropna()
y = df['y']
df.drop(['text', 'y'], axis=1, inplace=True)
df.head()

,cleaned_text
0,awww bummer shoulda got david carr third day
1,upset updat facebook text might cri result sch...
2,dive mani time ball manag save rest go bound
3,whole bodi feel itchi like fire
4,behav mad see


In [89]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, MaxPooling1D, Flatten, Bidirectional, GlobalMaxPool1D, SpatialDropout1D
from tensorflow.keras.models import Sequential
from sklearn.linear_model import LogisticRegression
import scipy

In [88]:
def Tokenize_MinMax(X_train, X_test):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, oov_token='<oov>')
    MS = MinMaxScaler()
    tokenizer.fit_on_texts(X_train)
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)
    X_train_seq = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH)
    X_test_seq = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH)
    X_train_seq = MS.fit_transform(X_train_seq)
    X_test_seq = MS.transform(X_test_seq)
    return MS, tokenizer, X_train_seq, X_test_seq

df_train, df_test, y_train, y_test = train_test_split(df, y, test_size=1-TRAIN_SIZE, random_state=42)
MS, tok, X_train, X_test = Tokenize_MinMax(df_train['cleaned_text'], df_test['cleaned_text'])


In [71]:
LR = LogisticRegression()
LR.fit(X_train, y_train)
score = LR.score(X_test, y_test)
score2 = LR.score(X_train, y_train)
print("Accuracy:", score)
print("Accuracy:", score2)

Accuracy: 0.5089872077940433
Accuracy: 0.513136116232251


In [66]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(Dense(10, input_dim=input_dim, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=512, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Epoch 1/10
2488/2488 [==============================] - 12s 5ms/step - loss: 47.5638 - accuracy: 0.4970 - val_loss: 2.4439 - val_accuracy: 0.4997
Epoch 2/10
2488/2488 [==============================] - 11s 5ms/step - loss: 2.1664 - accuracy: 0.4987 - val_loss: 1.7465 - val_accuracy: 0.5016
Epoch 3/10
2488/2488 [==============================] - 12s 5ms/step - loss: 1.8254 - accuracy: 0.5012 - val_loss: 1.5457 - val_accuracy: 0.5033
Epoch 4/10
2488/2488 [==============================] - 12s 5ms/step - loss: 1.6531 - accuracy: 0.5011 - val_loss: 1.5942 - val_accuracy: 0.5043
Epoch 5/10
2488/2488 [==============================] - 11s 5ms/step - loss: 1.5258 - accuracy: 0.5012 - val_loss: 1.5194 - val_accuracy: 0.5017
Epoch 6/10
2488/2488 [==============================] - 10s 4ms/step - loss: 1.4315 - accuracy: 0.5023 - val_loss: 1.2631 - val_accuracy: 0.4968
Epoch 7/10
2488/2488 [==============================] - 11s 4ms/step - loss: 1.3152 - accuracy: 0.5029 - val_loss: 1.0396 - val_a

In [72]:
vocab_size = len(tok.word_index) + 1
embedding_dim = 50
model = Sequential()
model.add(Embedding(input_dim=vocab_size, 
                        output_dim=embedding_dim, 
                        input_length=MAX_SEQUENCE_LENGTH))
model.add(GlobalMaxPool1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=1024, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Epoch 1/10
1244/1244 [==============================] - 117s 94ms/step - loss: 0.6932 - accuracy: 0.5006 - val_loss: 0.6931 - val_accuracy: 0.5017
Epoch 2/10
1244/1244 [==============================] - 110s 88ms/step - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6931 - val_accuracy: 0.4983
Epoch 3/10
1244/1244 [==============================] - 115s 92ms/step - loss: 0.6932 - accuracy: 0.4991 - val_loss: 0.6932 - val_accuracy: 0.4983
Epoch 4/10
 843/1244 [===================>..........] - ETA: 34s - loss: 0.6932 - accuracy: 0.4995

KeyboardInterrupt: 

In [78]:
GLOVE_EMB = 'glove.6B.300d.txt'
EMBEDDING_DIM = 300
LR = 1e-3
BATCH_SIZE = 1024
EPOCHS = 10
WORD_INDEX = tok.word_index

In [80]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_matrix = create_embedding_matrix(GLOVE_EMB, WORD_INDEX, EMBEDDING_DIM)

In [85]:
len(WORD_INDEX)

199242

In [83]:
len(embedding_matrix)

199243

In [82]:
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.2660520068459118

In [91]:
embedding_layer = Embedding(vocab_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding_sequences = embedding_layer(sequence_input)
x = SpatialDropout1D(0.2)(embedding_sequences)
x = Conv1D(64, 5, activation='relu')(x)
x = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(sequence_input, outputs)

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

model.compile(optimizer=Adam(learning_rate=LR), loss='binary_crossentropy',
              metrics=['accuracy'])
ReduceLROnPlateau = ReduceLROnPlateau(factor=0.1,
                                     min_lr = 0.01,
                                     monitor = 'val_loss',
                                     verbose = 1)

history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,
                    validation_data=(X_test, y_test), callbacks=[ReduceLROnPlateau])

Epoch 1/10
1244/1244 [==============================] - 622s 490ms/step - loss: 0.6932 - accuracy: 0.5003 - val_loss: 0.6932 - val_accuracy: 0.4983
Epoch 2/10
1244/1244 [==============================] - 618s 497ms/step - loss: 0.6932 - accuracy: 0.4998 - val_loss: 0.6932 - val_accuracy: 0.4983
Epoch 3/10
1244/1244 [==============================] - 666s 536ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.4983
Epoch 4/10
  43/1244 [>.............................] - ETA: 11:02 - loss: 0.6931 - accuracy: 0.5001

KeyboardInterrupt: 

In [ ]:
s, (at, al) = plt.subplots(2,1)
at.plot(history.history['accuracy'], c= 'b')
at.plot(history.history['val_accuracy'], c='r')
at.set_title('model accuracy')
at.set_ylabel('accuracy')
at.set_xlabel('epoch')
at.legend(['LSTM_train', 'LSTM_val'], loc='upper left')

al.plot(history.history['loss'], c='m')
al.plot(history.history['val_loss'], c='c')
al.set_title('model loss')
al.set_ylabel('loss')
al.set_xlabel('epoch')
al.legend(['train', 'val'], loc = 'upper left')